In [1]:
pip install netcdf4 wget pyproj

     |████████████████████████████████| 6.5MB 6.8MB/s 
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=35267e1c510c50542838e9d7b015322fa03193b68c9fc2c98d331dc7b7f00b99
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [3]:
import pandas as pd
import netCDF4
from pyproj import Transformer, Proj
import numpy as np
import datetime
import time
from google.colab import files
import wget
import os
import math

## Get Converted Lat Lon Reference values
One time execution and save is enough

In [4]:
# #h08v05
wget.download('https://calwildfire.thesofttools.com/h08v05_lat_lon_ref.nc')
ll = netCDF4.Dataset('h08v05_lat_lon_ref.nc', mode='r')
h08v05_lat = ll.variables['lat'][:][:].filled()
h08v05_lon = ll.variables['lon'][:][:].filled()

In [5]:
np.savez_compressed('ca_lfmc_lat_lon_ref.npz', h08v05_lat=h08v05_lat, h08v05_lon=h08v05_lon)
files.download('/content/ca_lfmc_lat_lon_ref.npz')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Get Available Latest LFMC valued from netcdf files

In [6]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1R6dCXTlJ5ut-5pJaeMY6wumzxNXIVdOd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1R6dCXTlJ5ut-5pJaeMY6wumzxNXIVdOd" -O fmc_c6_2016_h08v05.nc  && rm -rf /tmp/cookies.txt

--2021-05-19 18:18:06--  https://docs.google.com/uc?export=download&confirm=lTJ0&id=1R6dCXTlJ5ut-5pJaeMY6wumzxNXIVdOd
Resolving docs.google.com (docs.google.com)... 173.194.215.138, 173.194.215.101, 173.194.215.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.215.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-98-docs.googleusercontent.com/docs/securesc/8s1dku8uid3eto834f0cerlo6fo0oc27/ipecl1lq9mefnamgf5u1eu564hjog04m/1621448250000/10508343099502025234/06358392081889512681Z/1R6dCXTlJ5ut-5pJaeMY6wumzxNXIVdOd?e=download [following]
--2021-05-19 18:18:07--  https://doc-0g-98-docs.googleusercontent.com/docs/securesc/8s1dku8uid3eto834f0cerlo6fo0oc27/ipecl1lq9mefnamgf5u1eu564hjog04m/1621448250000/10508343099502025234/06358392081889512681Z/1R6dCXTlJ5ut-5pJaeMY6wumzxNXIVdOd?e=download
Resolving doc-0g-98-docs.googleusercontent.com (doc-0g-98-docs.googleusercontent.com)... 173.194.210.132, 2607:f8b0:400c:c0f::84
Conne

In [7]:
#h08v05
!ls -ltrh
nc = netCDF4.Dataset('fmc_c6_2016_h08v05.nc', mode='r')
time_var = nc.variables['time'][:].filled()
maxidx=time_var.size-1 #Latest available value will be last index
h08v05_mean=nc.variables['lfmc_mean'][maxidx][:][:].filled()
h08v05_stdv=nc.variables['lfmc_stdv'][maxidx][:][:].filled()

total 4.9G
drwxr-xr-x 1 root root 4.0K May  6 13:44 sample_data
-rw-r--r-- 1 root root 336M May 19 18:06 h08v05_lat_lon_ref.nc
-rw-r--r-- 1 root root  35M May 19 18:06 ca_lfmc_lat_lon_ref.npz
-rw-r--r-- 1 root root 4.5G May 19 18:19 fmc_c6_2016_h08v05.nc


In [8]:
if h08v05_mean.size==5760000 and h08v05_stdv.size==5760000:  
  np.savez_compressed('ca_lfmc_values.npz', h08v05_mean=h08v05_mean, h08v05_stdv=h08v05_stdv)
  files.download('/content/ca_lfmc_values.npz')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Function to extract best lfmc value
CELLS = 2400
VERTICAL_TILES = 18
HORIZONTAL_TILES = 36
EARTH_RADIUS = 6371007.181
EARTH_WIDTH = 2 * math.pi * EARTH_RADIUS
TILE_WIDTH = EARTH_WIDTH / HORIZONTAL_TILES
TILE_HEIGHT = TILE_WIDTH
CELL_SIZE = TILE_WIDTH / CELLS
MODIS_GRID = Proj(f'+proj=sinu +R={EARTH_RADIUS} +nadgrids=@null +wktext')
def lat_lon_to_modis(lat, lon):
    x, y = MODIS_GRID(lon, lat)
    h = (EARTH_WIDTH * .5 + x) / TILE_WIDTH
    v = -(EARTH_WIDTH * .25 + y - (VERTICAL_TILES - 0) * TILE_HEIGHT) / TILE_HEIGHT
    return 'h'+str(int(h))+'v'+str(int(v))

In [ ]:
lat_lon_to_modis(-32.0794, 116.0867)

'h27v12'

In [ ]:
def ca_lfmc(lat, lon):
  tile=lat_lon_to_modis(lat, lon)
  if tile=='h08v05':
    mean=-9999.9
    stdv=-9999.9
    diff = abs(lfmc_lat_lon_ref[tile+'_lat'] - lat) + abs(lfmc_lat_lon_ref[tile+'_lon'] - lon)
    #print(np.unravel_index(diff.argmin(), diff.shape))
    idx=np.unravel_index(np.argsort(diff.ravel()), diff.shape)
    print(idx[0][1])
    print(idx[1][1])
    for i in range(0,1024):
      mean=lfmc_values[tile+'_mean'][idx[0][i]][idx[1][i]]
      stdv=lfmc_values[tile+'_stdv'][idx[0][i]][idx[1][i]]
      #print(mean)
      if mean>-9999 and stdv>-9999:
        print(i)
        break;
    return {'lfmc_mean':mean, 'lfmc_stdv':stdv}
  else:
    return {'lfmc_mean':-9999, 'lfmc_stdv':-9999}

In [ ]:
wa_lfmc(-32.0794, 116.0867)
#wa_lfmc(-26.2542, 114.1132)
#wa_lfmc(-23.4814, 122.7868)

498
2008
0


{'lfmc_mean': 117.91, 'lfmc_stdv': 19.29133}